# 第23章 属性描述符

## 23.1 描述符示例：属性验证

### 23.1.1 描述符相关术语

- 描述符类：实现描述符协议的类。
- 托管类：把描述符实例声明为类属性的类。
- 描述符实例：描述符类的各个实例，声明为托管类的类属性。
- 托管实例：托管类的实例。
- 储存属性：托管实例中存储托管属性的属性。
- 托管属性：托管类中由描述符实例处理的公开属性，值存储在储存属性中。

### 23.1.2 LineItem类第3版：一个简单的描述符

In [1]:
class Quantity:  # <1>

    def __init__(self, storage_name):
        # 托管实例中用于存储值的储存属性的名称
        self.storage_name = storage_name

    def __set__(self, instance, value):
        # 为托管属性赋值
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

    def __get__(self, instance, owner):
        return instance.__dict__[self.storage_name]

In [2]:
class LineItem:
    # 描述符实例管理weight属性
    weight = Quantity('weight')
    # 描述符实例管理price属性
    price = Quantity('price')

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

In [5]:
try:
    truffle = LineItem('White truffle', 100, 0)
except ValueError as e:
    print(e)

price must be > 0


### 23.1.3 LineItem类第4版：为储存属性自动命名

目标：避免在描述符实例中重复输入属性名。

In [6]:
class Quantity:
    # owner是托管类，name是在owner的类主体中描述符实例赋给的那个属性名
    def __set_name__(self, owner, name):
        self.storage_name = name          

    def __set__(self, instance, value): 
        if value > 0:
            instance.__dict__[self.storage_name] = value
        else:
            msg = f'{self.storage_name} must be > 0'
            raise ValueError(msg)

In [7]:
class LineItem:
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

### 23.1.4 LineItem类第5版：一种新型描述符

需求：虚构的有机食物网店遇到了一个问题，有个商品的描述信息为空，导致无法下单。

In [11]:
import abc

class Validated(abc.ABC):

    def __set_name__(self, owner, name):
        self.storage_name = name

    def __set__(self, instance, value):
        value = self.validate(self.storage_name, value)  
        instance.__dict__[self.storage_name] = value

    @abc.abstractmethod
    def validate(self, name, value):
        """return validated value or raise ValueError"""

In [12]:
class Quantity(Validated):
    """数值大于0"""

    def validate(self, name, value):  # <1>
        if value <= 0:
            raise ValueError(f'{name} must be > 0')
        return value

In [13]:
class NonBlank(Validated):
    """字符串至少要包含一个非空字符"""

    def validate(self, name, value):
        value = value.strip()
        if not value:  # <2>
            raise ValueError(f'{name} cannot be blank')
        return value  # <3>

In [14]:
class LineItem:
    # 描述不能为空
    description = NonBlank()
    weight = Quantity()
    price = Quantity()

    def __init__(self, description, weight, price):
        self.description = description
        self.weight = weight
        self.price = price

    def subtotal(self):
        return self.weight * self.price

## 23.2 覆盖型描述符与非覆盖型描述符对比

- 覆盖型描述符：实现`__set__`方法的描述符都属于覆盖型描述符。
- 没有`__get__`方法的覆盖型描述符：当读取时，只要有同名的实例属性，描述符就会被覆盖。
- 非覆盖型描述符：没有实现`__set__`方法的描述符都属于非覆盖型描述符。

## 23.3 方法是描述符

In [15]:
import collections


class Text(collections.UserString):

    def __repr__(self):
        return 'Text({!r})'.format(self.data)

    def reverse(self):
        return self[::-1]

In [16]:
word = Text('forward')
word

Text('forward')

In [17]:
# 反向拼写单词
word.reverse()

Text('drawrof')

In [18]:
# 在类上调用方法相当于调用函数
Text.reverse(Text('backward'))

Text('drawkcab')

In [20]:
# 函数都是非覆盖型描述符
Text.reverse.__get__(word)

<bound method Text.reverse of Text('forward')>

## 23.4 描述符用法建议

- 使用property保持简单：内置property类创建的是实现了`__set__`和`__get__`方法的覆盖型描述符。
- 只读描述符必须有`__set__`方法。
- 用于验证的描述符可以只有`__set__`方法。
- 仅有`__get__`方法的描述符可以实现高效缓存：`@functools.cached_property`装饰器创建的就是非覆盖型描述符。
- 非特殊的方法可以被实例属性覆盖：函数和方法只实现了`__get__`方法，属于非覆盖型描述符。

## 23.5 杂谈

`self`的设计：
- 设计方法必须简单，对实现和接口来说都是如此。简单的实现比简单的接口更重要。简单是设计过程中最重要的考虑因素。
- 这种做法最初是由Modula-3语言提供。
- 除了要明确把self作为参数，限制必须通过self方法实例属性也备受批评。
- 如果讨厌Python要求显式使用self，想想JavaScript中隐式this那变幻莫测的语义，这样就感觉好多了。